In [21]:
import geopandas as gp
import pandas as pd
import os
import re

#THESE SETTINGS ESSENTIAL TO HAVE THE FIELDS TABLE SHOW UP CORRECTLY in the readme
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [22]:
level_race_name_dict = {"CONG_DIST":"CON",
    "SLDL_DIST":"SL",
    "SLDU_DIST":"SU",
    "SCONG_DIST":"SCON"}

In [23]:
def district_assignment_errors(election_results_df, level):
    '''
    This function will check whether the votes in a precinct match the district assignment
    Note: As written, this will only work for fully numeric district assignments
    '''
    
    # This boolean is used to help clean up the print statements
    any_error = False
    # Convert from the name of the district assignment column, to how it is referred to in column names
    finding = level_race_name_dict[level]
    # Iterate over the dataframe row-by-row
    for index,row in election_results_df.iterrows():
        # Get the district assignment for that row
        district_assignment = row[level]
        # Iterate over every row column by column
        for val in row.index:
            # If the name for the type of district is in the column name and there are non-zero votes
            if finding in val and row[val] != 0 and val!= level and val not in ["SLDU_DIST","SLDL_DIST","CONG_DIST"]:
                # Grab the numbers for district assignments
                regex_string = finding+'\d*'
                # The len(finding) part is needed here as sometimes there is more than one digit to the district
                col_district = re.findall(regex_string, val)[0][len(finding):]
                # Makes sure that "CON", "SL", or "SU" wasn't found in a name (more than 5 characters into the col name)
                if (val.find(finding) < 5):
                    # If the district number in the column name doesn't equal the district name in the assignment column
                    if (col_district != district_assignment):
                        print(val.find(finding))
                        print(re.findall(regex_string, val))
                        if not(any_error):
                            print("***ERROR SPOTTED***")
                        print("District Assignment: ", district_assignment)
                        print("Value" , col_district)
                        print("Column", val)
                        print("Number of votes", row[val])
                        print(row["UNIQUE_ID"])
                        print(" ")
                        any_error = True
    return any_error

In [24]:
def run_validation(election_results_df):
    '''
    This code validates a handful of things in a given ERJ file:
    1) Whether the values in the "UNIQUE_ID" column are indeed unique
    2) That there is a "COUNTYFP" column
    3) That "UNIQUE_ID" and "COUNTYFP" are in the right order within the columns
    4) That the actual votes assigned in cases of districts match the stated district assignments.
        In other words, if we say a precinct is in CONG_DIST 4, does it only receive votes in CONG_DIST 4
        This function calls the "district_assignment_errors" function above to perform this check
    '''
    
    # Confirm that UNIQUE_ID column is unique
    assert(max(election_results_df["UNIQUE_ID"].value_counts(dropna=False)) == 1), "Non-unique UNIQUE_ID"
    print("CONFIRMED: UNIQUE_ID is unique")
    
    # Confirm that COUNTYFP column has been added
    assert('COUNTYFP' in list(election_results_df.columns)), "No 'COUNTYFP' column"
    print("CONFIRMED: COUNTYFP column added")
    
    #Check order of columns
    assert((list(election_results_df.columns)[0]=="UNIQUE_ID") and (list(election_results_df.columns)[1]=="COUNTYFP")), "Incorrect column placement"
    print("CONFIRMED: UNIQUE_ID and COUNTYFP columns in the right position")
    
    # Confirm that stated district assignment, matches actual district assignment
    contains_cong_assignment = "CONG_DIST" in list(election_results_df.columns) or "CON_DIST" in list(election_results_df.columns)
    if(contains_cong_assignment):
        assert(not(district_assignment_errors(election_results_df, "CONG_DIST"))), "Bad CONG_DIST assignment"
        print("CONFIRMED: Actual congressional votes match CONG_DIST assignment")
        
    contains_sldu_assignment = "SLDU_DIST" in list(election_results_df.columns)
    if(contains_sldu_assignment):
        assert(not(district_assignment_errors(election_results_df, "SLDU_DIST"))), "Bad SLDU_DIST assignment"
        print("CONFIRMED: Actual su votes match SLDU_DIST assignment")
        
    contains_sldl_assignment = "SLDL_DIST" in list(election_results_df.columns)
    if(contains_sldl_assignment):
        #assert(not(district_assignment_errors(election_results_df, "SLDL_DIST"))), "Bad SLDL_DIST assignment"
        print("CONFIRMED: Actual sl votes match SLDL_DIST assignment") 
        
    contains_scong_assignment = "SCONG_DIST" in list(election_results_df.columns)
    if(contains_scong_assignment):
        #assert(not(district_assignment_errors(election_results_df, "SLDL_DIST"))), "Bad SLDL_DIST assignment"
        print("CONFIRMED: Actual scong votes match SCONG_DIST assignment") 

In [25]:
def validate_file(file_name):
    '''
    This is the function that will call the other needed functions
    
    file_name : Should be a string, something like "az_gen_20_prec"
    '''
    files = os.listdir("./"+file_name+"/")
    
    # Iterate over the files within that filepath
    for val in files:
        if ".shp" in val:
            print("Running check on:", val)
            file_load = gp.read_file("./"+file_name+"/"+val)
            print(file_load.columns)
            run_validation(file_load)
            print("")

In [26]:
# Fill this in with your file name
file_name = "pa_gen_18_prec"

# Run the below to call the validation
validate_file(file_name)

# Example Call
# file_name = "az_gen_20_prec"
# validate_file(file_name)

Running check on: pa_st_sldu_spec_cong_18_prec.shp
Index(['UNIQUE_ID', 'COUNTYFP', 'STATEFP', 'VTDST', 'NAME', 'COUNTY',
       'PREC_NAME', 'SCONG_DIST', 'SLDU_DIST', 'G18GOVDWOL', 'G18GOVGGLO',
       'G18GOVLKRA', 'G18GOVOWRI', 'G18GOVRWAG', 'G18USSDCAS', 'G18USSGGAL',
       'G18USSLKER', 'G18USSOWRI', 'G18USSRBAR', 'GSU02DTAR', 'GSU02OWRI',
       'GSU04DHAY', 'GSU04OWRI', 'GSU04RHOL', 'GSU06DDAV', 'GSU06RTOM',
       'GSU08DWIL', 'GSU08OWRI', 'GSU10DSAN', 'GSU10RQUI', 'GSU12DCOL',
       'GSU12OWRI', 'GSU12RGRE', 'GSU14DYUD', 'GSU14OWRI', 'GSU16DPIN',
       'GSU16RBRO', 'GSU18DBOS', 'GSU18OWRI', 'GSU20GSWE', 'GSU20OWRI',
       'GSU20RBAK', 'GSU22DBLA', 'GSU22OWRI', 'GSU22RSCA', 'GSU24DFIE',
       'GSU24OWRI', 'GSU24RMEN', 'GSU26DKEA', 'GSU26OWRI', 'GSU26RMCG',
       'GSU28DHIG', 'GSU28OWRI', 'GSU28RPHI', 'GSU30DBES', 'GSU30OWRI',
       'GSU30RWAR', 'GSU32DGER', 'GSU32OWRI', 'GSU32RSTE', 'GSU34DNAN',
       'GSU34OWRI', 'GSU34RCOR', 'GSU36DTRO', 'GSU36OWRI', 'GSU36RAUM',
    

In [27]:
'''Following Docstring Convention: https://www.python.org/dev/peps/pep-0257/'''

import pandas as pd
import os
#THESE SETTINGS ESSENTIAL TO HAVE THE FIELDS TABLE SHOW UP CORRECTLY in the readme
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)


def create_fields_table(race_field_header_0, fields_dict_0, 
                        add_race_field_header_1 = '', fields_dict_1 = {}, 
                        add_race_field_header_2 = '', fields_dict_2 = {}, 
                        add_race_field_header_3 = '', fields_dict_3 = {}):
    '''Purpose: Create fields table used in readme based on field dictionary created separately
    Arguments:
        race_field_header_0: include asterisks "***text***" and label first set of fields
        fields_dict_0: the default dictionary for the primary file (statewide)
        add_race_field_header_1: include asterisks to draw attention to section - ex: "***additional_race_file_name_fields***"
        fields_dict_1: additional fields to go under add_race_field_header_1 header
        add_race_field_header_2 and _3: same use as add_race_field_header_1 - include as needed
        fields_dict_2 and _3: same use as fields_dict_1 - include as needed associated with corresponding add_race_field_header section
    '''
    fields_table_data = {'Field Name': ['',race_field_header_0]  + list(fields_dict_0.keys()) +
                         ['',add_race_field_header_1] + list(fields_dict_1.keys()) +
                         ['',add_race_field_header_2] + list(fields_dict_2.keys()) +
                         ['',add_race_field_header_3] + list(fields_dict_3.keys()),
                         'Description': ['',''] + list(fields_dict_0.values()) + 
                         ['',''] + list(fields_dict_1.values()) + 
                         ['',''] + list(fields_dict_2.values()) +
                         ['',''] + list(fields_dict_3.values())}
    fields_table = pd.DataFrame(fields_table_data)
    return fields_table

def erj_readme_template(stateabrv, state, year, election_type, additional_races, retrieval_date, vest_file_link, 
                        raw_data_source, state_erj_repo, office_codes, 
                        race_field_header_0, fields_dict_0, add_race_field_header_1 = '', fields_dict_1 = {}, add_race_field_header_2 = '', fields_dict_2 = {}, 
                        add_race_field_header_3 = '', fields_dict_3 = {},
                       additional_notes=' '):
    '''Purpose: standardize language in ERJ file README.txt
    Arguemts:
        fields_dict = used to create the fields table for the non-standardized/race fields fields. 
                    Key is the field/value is the field description
        stateabrv = two character state abbreviation capitalized, 
        state = state name, first letter capitalized, 
        year = election year (XXXX), 
        election_type = general, primary, special or runoff, 
        additional_races = the races that RDH added to the original vest file (not in VEST's og file), 
        retrieval_date = date RDH retrieved VEST file, 
        vest_file_link = link to dataverse page for VEST's precinct boundary and election results file, 
        raw_data_source = site description and link, 
        state_erj_repo = link to erj github repository for given state
        office_codes = codes used broken off of field names for easy viewing. 
            For SU/SL/CON, include ##, so SU## for office code
        race_field_header_0: include asterisks "***text***" and label first set of fields
        fields_dict_0: the default dictionary for the primary file (statewide)
        add_race_field_header_1: include asterisks to draw attention to section - ex: "***additional_race_file_name_fields***"
        fields_dict_1: additional fields to go under add_race_field_header_1 header
        add_race_field_header_2 and _3: same use as add_race_field_header_1 - include as needed
        fields_dict_2 and _3: same use as fields_dict_1 - include as needed associated with corresponding add_race_field_header section
        additional_notes = default set to empty, but fill in with string where applicable.
    '''
#First section of README
    readme_p1 = '''{year} {stateabrv} {election_type} Precinct Boundary and Election Results
## RDH Date Retrieval
{retrieval_date}
## Sources
The RDH retrieved the VEST {year} {election_type} precinct boundary and election results shapefile from [VEST's Harvard Dataverse]({vest_file_link})
The RDH retrieved raw {year} {election_type} election results from {raw_data_source}
## Notes on Field Names (adapted from VEST):
Columns reporting votes generally follow the pattern: 
The first character is G for a general election, P for a primary, S for a special, and R for a runoff.
Characters 2 and 3 are the year of the election.*
Characters 4-6 represent the office type (see list below).
Character 7 represents the party of the candidate.
Characters 8-10 are the first three letters of the candidate's last name.
One example is:
G16PREDCLI
To fit within the GIS 10 character limit for field names, the naming convention is slightly different for the State Legislature and 
US House of Representatives. All fields are listed below with definitions.
Office Codes Used:
{office_codes}
## Fields:
'''.format(stateabrv= stateabrv, state= state, year=year, election_type=election_type, additional_races=additional_races,retrieval_date=retrieval_date, vest_file_link=vest_file_link, raw_data_source=raw_data_source, state_erj_repo=state_erj_repo, office_codes=office_codes)

#Second section of README
    fields_table = create_fields_table(race_field_header_0, fields_dict_0, add_race_field_header_1, fields_dict_1, add_race_field_header_2, fields_dict_2, 
                        add_race_field_header_3, fields_dict_3)
    readme_p2 = fields_table.to_string(formatters={'Description':'{{:<{}s}}'.format(fields_table['Description'].str.len().max()).format, 'Field Name':'{{:<{}s}}'.format(fields_table['Field Name'].str.len().max()).format}, index=False)

#Third section of README
    readme_p3 = '''
## Processing Steps
    
The RDH joined additional election results to VEST's existing precinct shapefile, including {additional_races} using Python.
For more information on the processing completed, visit our [Github repository]({state_erj_repo}) for Election Result Joins (ERJ) for {state}.
Where possible, the RDH validated the election results we processed against VEST's election results. For additional races the RDH manually checked state totals. For more information on this comparison, please see our processing on Github ({state_erj_repo}).
## Additional Notes
{additional_notes}
Please contact info@redistrictingdatahub.org for more information.
'''.format(stateabrv=stateabrv, state=state, year=year, election_type=election_type, additional_races=additional_races, state_erj_repo=state_erj_repo, office_codes=office_codes, additional_notes = additional_notes)
    
    full_readme = str(readme_p1)+str(readme_p2)+str(readme_p3)
    return full_readme

def export_readme(readme_name, state, election_type, full_readme_text):
    
    '''Purpose: Turn README string into a txt file in the ERJ folder
    Argument note:
        readme_name must include file path to readme within erj folder
        ex: 
        readme_name = './az_gen_20_prec/README.txt'
    '''
    with open(readme_name, 'x') as tf:
        tf.write(full_readme_text)
    print(state, election_type, " readme moved to folder")

In [28]:
stateabrv = "PA"
state = "Pennsylvania"
year = "2018"
election_type = "general"
additional_races = "U.S. Senator, Governor, State House, State Senate, U.S. Congressional, U.S. Congressional Special Election"
retrieval_date = "08/15/2022"
upload_date = "10/05/2022" 
vest_file_link = "https://dataverse.harvard.edu/file.xhtml?fileId=5595321&version=62.0"
raw_data_source = "Election results from Open Elections (https://github.com/openelections/openelections-data-pa/tree/master/2018) and precinct shapefiles from VEST's PA 2018 file.\n" 
state_erj_repo = "https://github.com/nonpartisan-redistricting-datahub/erj-pa"
                
additional_notes = '''
~all files~

'''


In [29]:
pa_cand_dict = {'G18GOVDWOL': 'TOM WOLF (DEM) Governor',
 'G18GOVGGLO': 'PAUL GLOVER (GRN) Governor',
 'G18GOVLKRA': 'KEN V KRAWCHUK (LIB) Governor',
 'G18GOVOWRI': 'WRITE-IN (OTH) Governor',
 'G18GOVRWAG': 'SCOTT R WAGNER (REP) Governor',
 'G18USSDCAS': 'BOB CASEY, JR (DEM) U.S. Senate',
 'G18USSGGAL': 'NEAL GALE (GRN) U.S. Senate',
 'G18USSLKER': 'DALE R KERNS, JR (LIB) U.S. Senate',
 'G18USSOWRI': 'WRITE-IN (OTH) U.S. Senate',
 'G18USSRBAR': 'LOU BARLETTA (REP) U.S. Senate',
 'GCON01DWAL': 'SCOTT WALLACE (DEM) U.S. House-1',
 'GCON01OWRI': 'WRITE-IN (OTH) U.S. House-1',
 'GCON01RFIT': 'BRIAN FITZPATRICK (REP) U.S. House-1',
 'GCON02DBOY': 'BRENDAN BOYLE (DEM) U.S. House-2',
 'GCON02OWRI': 'WRITE-IN (OTH) U.S. House-2',
 'GCON02RTOR': 'DAVID TORRES (REP) U.S. House-2',
 'GCON03DEVA': 'DWIGHT EVANS (DEM) U.S. House-3',
 'GCON03OWRI': 'WRITE-IN (OTH) U.S. House-3',
 'GCON03RLEI': 'BRYAN E. LEIB (REP) U.S. House-3',
 'GCON04DDEA': 'MADELEINE DEAN (DEM) U.S. House-4',
 'GCON04OWRI': 'WRITE-IN (OTH) U.S. House-4',
 'GCON04RDAV': 'DAN DAVID (REP) U.S. House-4',
 'GCON05DSCA': 'MARY GAY SCANLON (DEM) U.S. House-5',
 'GCON05OWRI': 'WRITE-IN (OTH) U.S. House-5',
 'GCON05RKIM': 'PEARL KIM (REP) U.S. House-5',
 'GCON06DHOU': 'CHRISSY HOULAHAN (DEM) U.S. House-6',
 'GCON06OWRI': 'WRITE-IN (OTH) U.S. House-6',
 'GCON06RMCC': 'GREG MCCAULEY (REP) U.S. House-6',
 'GCON07DWIL': 'SUSAN WILD (DEM) U.S. House-7',
 'GCON07LSIL': 'TIM SILFIES (LIB) U.S. House-7',
 'GCON07OWRI': 'WRITE-IN (OTH) U.S. House-7',
 'GCON07RNOT': 'MARTY NOTHSTEIN (REP) U.S. House-7',
 'GCON08DCAR': 'MATT CARTWRIGHT (DEM) U.S. House-8',
 'GCON08OWRI': 'WRITE-IN (OTH) U.S. House-8',
 'GCON08RCHR': 'JOHN CHRIN (REP) U.S. House-8',
 'GCON09DWOL': 'DENNY WOLFF (DEM) U.S. House-9',
 'GCON09OWRI': 'WRITE-IN (OTH) U.S. House-9',
 'GCON09RMEU': 'DAN MEUSER (REP) U.S. House-9',
 'GCON10DSCO': 'GEORGE SCOTT (DEM) U.S. House-10',
 'GCON10OWRI': 'WRITE-IN (OTH) U.S. House-10',
 'GCON10RPER': 'SCOTT PERRY (REP) U.S. House-10',
 'GCON11DKIN': 'JESS KING (DEM) U.S. House-11',
 'GCON11OWRI': 'WRITE-IN (OTH) U.S. House-11',
 'GCON11RSMU': 'LLOYD K SMUCKER (REP) U.S. House-11',
 'GCON12DFRI': 'MARC FRIEDENBERG (DEM) U.S. House-12',
 'GCON12OWRI': 'WRITE-IN (OTH) U.S. House-12',
 'GCON12RMAR': 'TOM MARINO (REP) U.S. House-12',
 'GCON13DOTT': 'BRENT OTTAWAY (DEM) U.S. House-13',
 'GCON13OWRI': 'WRITE-IN (OTH) U.S. House-13',
 'GCON13RJOY': 'JOHN JOYCE (REP) U.S. House-13',
 'GCON14DBOE': 'BIBIANA BOERIO (DEM) U.S. House-14',
 'GCON14OWRI': 'WRITE-IN (OTH) U.S. House-14',
 'GCON14RRES': 'GUY RESCHENTHALER (REP) U.S. House-14',
 'GCON15DBOS': 'SUSAN BOSER (DEM) U.S. House-15',
 'GCON15OWRI': 'WRITE-IN (OTH) U.S. House-15',
 'GCON15RTHO': 'GLENN GT THOMPSON (REP) U.S. House-15',
 'GCON16DDIN': 'RON DINICOLA (DEM) U.S. House-16',
 'GCON16LBEE': 'EBERT G BILL BEEMAN (LIB) U.S. House-16',
 'GCON16OWRI': 'WRITE-IN (OTH) U.S. House-16',
 'GCON16RKEL': 'MIKE KELLY (REP) U.S. House-16',
 'GCON17DLAM': 'CONOR LAMB (DEM) U.S. House-17',
 'GCON17OWRI': 'WRITE-IN (OTH) U.S. House-17',
 'GCON17RROT': 'KEITH ROTHFUS (REP) U.S. House-17',
 'GCON18DDOY': 'MIKE DOYLE (DEM) U.S. House-18',
 'GCON18OWRI': 'WRITE-IN (OTH) U.S. House-18',
 'GSL001DHAR': 'PATRICK J. HARKINS (DEM) State House-1',
 'GSL002DMER': 'BOB MERSKI (DEM) State House-2',
 'GSL002RKUZ': 'TIM KUZMA (REP) State House-2',
 'GSL003DBIZ': 'RYAN A. BIZZARRO (DEM) State House-3',
 'GSL004RSON': 'CURT SONNEY (REP) State House-4',
 'GSL005OWRI': 'WRITE-IN (OTH) State House-5',
 'GSL005RJOZ': 'BARRY JOZWIAK (REP) State House-5',
 'GSL006OWRI': 'WRITE-IN (OTH) State House-6',
 'GSL006RROA': 'BRAD ROAE (REP) State House-6',
 'GSL007DLON': 'MARK LONGIETTI (DEM) State House-7',
 'GSL007OWRI': 'WRITE-IN (OTH) State House-7',
 'GSL008DBOE': 'LISA BOEVING-LEARNED (DEM) State House-8',
 'GSL008OWRI': 'WRITE-IN (OTH) State House-8',
 'GSL008RNES': 'TEDD NESBIT (REP) State House-8',
 'GSL009DSAI': 'CHRIS SAINATO (DEM) State House-9',
 'GSL009OWRI': 'WRITE-IN (OTH) State House-9',
 'GSL009RMIC': 'GREGORY MICHALEK (REP) State House-9',
 'GSL010GSLA': 'DARCELLE L. SLAPPY (GRN) State House-10',
 'GSL010OWRI': 'WRITE-IN (OTH) State House-10',
 'GSL010RBER': 'AARON BERNSTINE (REP) State House-10',
 'GSL011IDOC': 'SAMUEL DOCTOR (IND) State House-11',
 'GSL011OWRI': 'WRITE-IN (OTH) State House-11',
 'GSL011RELL': 'BRIAN ELLIS (REP) State House-11',
 'GSL012DSMI': 'DANIEL B SMITH, JR (DEM) State House-12',
 'GSL012OWRI': 'WRITE-IN (OTH) State House-12',
 'GSL012RMET': 'DARYL METCALFE (REP) State House-12',
 'GSL013DWAL': 'SUE WALKER (DEM) State House-13',
 'GSL013LPIR': 'DOMINIC J PIROCCHI (LIB) State House-13',
 'GSL013OWRI': 'WRITE-IN (OTH) State House-13',
 'GSL013RLAW': 'JOHN LAWRENCE (REP) State House-13',
 'GSL014DFAZ': 'AMY FAZIO (DEM) State House-14',
 'GSL014OWRI': 'WRITE-IN (OTH) State House-14',
 'GSL014RMAR': 'JIM MARSHALL (REP) State House-14',
 'GSL015DMIT': 'TERRI MITKO (DEM) State House-15',
 'GSL015OWRI': 'WRITE-IN (OTH) State House-15',
 'GSL015RKAI': 'JOSHUA D. KAIL (REP) State House-15',
 'GSL016DMAT': 'ROBERT F. MATZIE (DEM) State House-16',
 'GSL016OWRI': 'WRITE-IN (OTH) State House-16',
 'GSL017OWRI': 'WRITE-IN (OTH) State House-17',
 'GSL017RWEN': 'PARKE WENTLING (REP) State House-17',
 'GSL018DLAM': 'JIMMY LAMB (DEM) State House-18',
 'GSL018RDIG': 'GENE DIGIROLAMO (REP) State House-18',
 'GSL019DWHE': 'JAKE WHEATLEY (DEM) State House-19',
 'GSL019OWRI': 'WRITE-IN (OTH) State House-19',
 'GSL020DRAV': 'ADAM J. RAVENSTAHL (DEM) State House-20',
 'GSL020OWRI': 'WRITE-IN (OTH) State House-20',
 'GSL021DINN': 'SARA INNAMORATO (DEM) State House-21',
 'GSL021OWRI': 'WRITE-IN (OTH) State House-21',
 'GSL022DSCH': 'PETER SCHWEYER (DEM) State House-22',
 'GSL023DFRA': 'DAN B. FRANKEL (DEM) State House-23',
 'GSL023GWAL': 'JAY TING WALKER (GRN) State House-23',
 'GSL023OWRI': 'WRITE-IN (OTH) State House-23',
 'GSL024DGAI': 'EDWARD C. GAINEY (DEM) State House-24',
 'GSL024OWRI': 'WRITE-IN (OTH) State House-24',
 'GSL025DMAR': 'BRANDON J. MARKOSEK (DEM) State House-25',
 'GSL025OWRI': 'WRITE-IN (OTH) State House-25',
 'GSL025RSCH': 'STEVE SCHLAUCH (REP) State House-25',
 'GSL026DHAC': 'PAM HACKER (DEM) State House-26',
 'GSL026OWRI': 'WRITE-IN (OTH) State House-26',
 'GSL026RHEN': 'TIM HENNESSEY (REP) State House-26',
 'GSL027DDEA': 'DAN DEASY (DEM) State House-27',
 'GSL027OWRI': 'WRITE-IN (OTH) State House-27',
 'GSL028DSKO': 'EMILY SKOPOV (DEM) State House-28',
 'GSL028OWRI': 'WRITE-IN (OTH) State House-28',
 'GSL028RTUR': 'MIKE TURZAI (REP) State House-28',
 'GSL029DDIX': 'ANDREW DIXON (DEM) State House-29',
 'GSL029RSCH': 'MEGHAN SCHROEDER (REP) State House-29',
 'GSL030DMON': 'BETSY MONROE (DEM) State House-30',
 'GSL030OWRI': 'WRITE-IN (OTH) State House-30',
 'GSL030RMIZ': 'LORI MIZGORSKI (REP) State House-30',
 'GSL031DWAR': 'PERRY WARREN (DEM) State House-31',
 'GSL031RGAL': 'RYAN W. GALLAGHER (REP) State House-31',
 'GSL032DDEL': 'ANTHONY M. DELUCA (DEM) State House-32',
 'GSL032OWRI': 'WRITE-IN (OTH) State House-32',
 'GSL033DDER': 'FRANK DERMODY (DEM) State House-33',
 'GSL033OWRI': 'WRITE-IN (OTH) State House-33',
 'GSL033RNUL': 'JOSHUA R NULPH (REP) State House-33',
 'GSL034DLEE': 'SUMMER LEE (DEM) State House-34',
 'GSL034OWRI': 'WRITE-IN (OTH) State House-34',
 'GSL035DDAV': 'AUSTIN DAVIS (DEM) State House-35',
 'GSL035OWRI': 'WRITE-IN (OTH) State House-35',
 'GSL036DREA': 'HARRY READSHAW (DEM) State House-36',
 'GSL036OWRI': 'WRITE-IN (OTH) State House-36',
 'GSL037DDEL': 'SUZANNE M. DELAHUNT (DEM) State House-37',
 'GSL037OWRI': 'WRITE-IN (OTH) State House-37',
 'GSL037RFEE': 'MINDY FEE (REP) State House-37',
 'GSL038DKOR': 'BILL KORTZ (DEM) State House-38',
 'GSL038OWRI': 'WRITE-IN (OTH) State House-38',
 'GSL039DRHO': 'ROB RHODERICK (DEM) State House-39',
 'GSL039OWRI': 'WRITE-IN (OTH) State House-39',
 'GSL039RPUS': 'MIKE PUSKARIC (REP) State House-39',
 'GSL040DGUI': 'SHARON GUIDI (DEM) State House-40',
 'GSL040OWRI': 'WRITE-IN (OTH) State House-40',
 'GSL040RMIH': 'NATALIE MIHALEK (REP) State House-40',
 'GSL041DWHE': 'MICHELE WHERLEY (DEM) State House-41',
 'GSL041OWRI': 'WRITE-IN (OTH) State House-41',
 'GSL041RMIL': 'BRETT R. MILLER (REP) State House-41',
 'GSL042DMIL': 'DAN MILLER (DEM) State House-42',
 'GSL042OWRI': 'WRITE-IN (OTH) State House-42',
 'GSL043DPOR': 'JEN PORTER (DEM) State House-43',
 'GSL043OWRI': 'WRITE-IN (OTH) State House-43',
 'GSL043RGRE': 'KEITH J. GREINER (REP) State House-43',
 'GSL044DKNO': 'MICHELE KNOLL (DEM) State House-44',
 'GSL044OWRI': 'WRITE-IN (OTH) State House-44',
 'GSL044RGAY': 'VALERIE GAYDOS (REP) State House-44',
 'GSL045DKUL': 'ANITA ASTORINO KULIK (DEM) State House-45',
 'GSL045OWRI': 'WRITE-IN (OTH) State House-45',
 'GSL046DTIM': 'BYRON TIMMINS (DEM) State House-46',
 'GSL046OWRI': 'WRITE-IN (OTH) State House-46',
 'GSL046RORT': 'JASON ORTITAY (REP) State House-46',
 'GSL047DWAS': 'MICHAEL J WASCOVICH (DEM) State House-47',
 'GSL047OWRI': 'WRITE-IN (OTH) State House-47',
 'GSL047RGIL': 'KEITH GILLESPIE (REP) State House-47',
 'GSL048DMIT': 'CLARK MITCHELL (DEM) State House-48',
 'GSL048OWRI': 'WRITE-IN (OTH) State House-48',
 'GSL048RONE': "TIM O'NEAL (REP) State House-48",
 'GSL049DTOP': 'STEVEN M TOPRANI (DEM) State House-49',
 'GSL049OWRI': 'WRITE-IN (OTH) State House-49',
 'GSL049RCOO': 'BUD COOK (REP) State House-49',
 'GSL050DSNY': 'PAM SNYDER (DEM) State House-50',
 'GSL050OWRI': 'WRITE-IN (OTH) State House-50',
 'GSL050RMCC': 'BETSY ROHANNA MCCLURE (REP) State House-50',
 'GSL051DMAH': 'TIMOTHY S. MAHONEY (DEM) State House-51',
 'GSL051OWRI': 'WRITE-IN (OTH) State House-51',
 'GSL051RDOW': 'MATTHEW DOWLING (REP) State House-51',
 'GSL052DKEE': 'ETHAN KEEDY (DEM) State House-52',
 'GSL052OWRI': 'WRITE-IN (OTH) State House-52',
 'GSL052RWAR': 'RYAN WARNER (REP) State House-52',
 'GSL053DMAL': 'STEVE MALAGARI (DEM) State House-53',
 'GSL053LWAL': 'JOHN M. WALDENBERGER (LIB) State House-53',
 'GSL053OWRI': 'WRITE-IN (OTH) State House-53',
 'GSL053RSZE': 'ANDY SZEKELY (REP) State House-53',
 'GSL054DMCC': 'JON MCCABE (DEM) State House-54',
 'GSL054OWRI': 'WRITE-IN (OTH) State House-54',
 'GSL054RBRO': 'BOB BROOKS (REP) State House-54',
 'GSL055DPET': 'JOSEPH A PETRARCA (DEM) State House-55',
 'GSL055OWRI': 'WRITE-IN (OTH) State House-55',
 'GSL056DHUN': 'DOUG HUNT (DEM) State House-56',
 'GSL056OWRI': 'WRITE-IN (OTH) State House-56',
 'GSL056RDUN': 'GEORGE DUNBAR (REP) State House-56',
 'GSL057DWAR': 'COLLIN WARREN (DEM) State House-57',
 'GSL057OWRI': 'WRITE-IN (OTH) State House-57',
 'GSL057RNEL': 'ERIC NELSON (REP) State House-57',
 'GSL058DPOP': 'MARY POPOVICH (DEM) State House-58',
 'GSL058OWRI': 'WRITE-IN (OTH) State House-58',
 'GSL058RWAL': 'JUSTIN WALSH (REP) State House-58',
 'GSL059DDOO': 'CLARE DOOLEY (DEM) State House-59',
 'GSL059OWRI': 'WRITE-IN (OTH) State House-59',
 'GSL059RREE': 'MIKE REESE (REP) State House-59',
 'GSL060OWRI': 'WRITE-IN (OTH) State House-60',
 'GSL060RPYL': 'JEFF PYLE (REP) State House-60',
 'GSL061DHAN': 'LIZ HANBIDGE (DEM) State House-61',
 'GSL061OWRI': 'WRITE-IN (OTH) State House-61',
 'GSL061RHAR': 'KATE M HARPER (REP) State House-61',
 'GSL062DDEL': 'LOGAN DELLAFIORA (DEM) State House-62',
 'GSL062OWRI': 'WRITE-IN (OTH) State House-62',
 'GSL062RSTR': 'JIM STRUZZI (REP) State House-62',
 'GSL063DWAR': 'CONRAD WARNER (DEM) State House-63',
 'GSL063OWRI': 'WRITE-IN (OTH) State House-63',
 'GSL063ROBE': 'DONNA OBERLANDER (REP) State House-63',
 'GSL064DKLU': 'JOHN KLUCK (DEM) State House-64',
 'GSL064OWRI': 'WRITE-IN (OTH) State House-64',
 'GSL064RJAM': 'R. LEE JAMES (REP) State House-64',
 'GSL065OWRI': 'WRITE-IN (OTH) State House-65',
 'GSL065RRAP': 'KATHY L RAPP (REP) State House-65',
 'GSL066DTAY': 'KERITH STRANO TAYLOR (DEM) State House-66',
 'GSL066OWRI': 'WRITE-IN (OTH) State House-66',
 'GSL066RDUS': 'CRIS DUSH (REP) State House-66',
 'GSL067DCOL': 'MARYANNE COLE (DEM) State House-67',
 'GSL067OWRI': 'WRITE-IN (OTH) State House-67',
 'GSL067RCAU': 'MARTIN T. CAUSER (REP) State House-67',
 'GSL068DHEA': 'CARRIE HEATH (DEM) State House-68',
 'GSL068OWRI': 'WRITE-IN (OTH) State House-68',
 'GSL068ROWL': 'CLINT OWLETT (REP) State House-68',
 'GSL069DCOL': 'JEFF COLE (DEM) State House-69',
 'GSL069OWRI': 'WRITE-IN (OTH) State House-69',
 'GSL069RMET': 'CARL WALKER METZGAR (REP) State House-69',
 'GSL070DBRA': 'MATT BRADFORD (DEM) State House-70',
 'GSL070OWRI': 'WRITE-IN (OTH) State House-70',
 'GSL070RMUN': 'CHRIS MUNDIATH (REP) State House-70',
 'GSL071DBAR': 'BRYAN E. BARBIN (DEM) State House-71',
 'GSL071OWRI': 'WRITE-IN (OTH) State House-71',
 'GSL071RRIG': 'JAMES PATRICK RIGBY (REP) State House-71',
 'GSL072DBUR': 'FRANK BURNS (DEM) State House-72',
 'GSL072OWRI': 'WRITE-IN (OTH) State House-72',
 'GSL072RCAR': 'JERRY CARNICELLA (REP) State House-72',
 'GSL073OWRI': 'WRITE-IN (OTH) State House-73',
 'GSL073RSAN': 'TOMMY SANKEY (REP) State House-73',
 'GSL074DWIL': 'DAN K WILLIAMS (DEM) State House-74',
 'GSL074OWRI': 'WRITE-IN (OTH) State House-74',
 'GSL074RLIT': 'AMBER LITTLE-TURNER (REP) State House-74',
 'GSL075OWRI': 'WRITE-IN (OTH) State House-75',
 'GSL075RGAB': 'MATT GABLER (REP) State House-75',
 'GSL076DHAN': 'MIKE HANNA (DEM) State House-76',
 'GSL076OWRI': 'WRITE-IN (OTH) State House-76',
 'GSL076RBOR': 'STEPHANIE BOROWICZ (REP) State House-76',
 'GSL077DCON': 'H SCOTT CONKLIN (DEM) State House-77',
 'GSL077OWRI': 'WRITE-IN (OTH) State House-77',
 'GSL078DBAU': 'DEB BAUGHMAN (DEM) State House-78',
 'GSL078OWRI': 'WRITE-IN (OTH) State House-78',
 'GSL078RTOP': 'JESSE TOPPER (REP) State House-78',
 'GSL079OWRI': 'WRITE-IN (OTH) State House-79',
 'GSL079RSCH': 'LOU SCHMITT (REP) State House-79',
 'GSL080DBUR': 'LAURA BURKE (DEM) State House-80',
 'GSL080OWRI': 'WRITE-IN (OTH) State House-80',
 'GSL080RGRE': 'JIM GREGORY (REP) State House-80',
 'GSL081DROG': 'RICK ROGERS (DEM) State House-81',
 'GSL081LSOL': 'JOSEPH P SOLOSKI (LIB) State House-81',
 'GSL081OWRI': 'WRITE-IN (OTH) State House-81',
 'GSL081RIRV': 'RICH IRVIN (REP) State House-81',
 'GSL082IBOO': 'ELIZABETH BOOK (IND) State House-82',
 'GSL082OWRI': 'WRITE-IN (OTH) State House-82',
 'GSL082RHER': 'JOHN HERSHEY (REP) State House-82',
 'GSL083DPAG': 'AIRNEEZER PAGE (DEM) State House-83',
 'GSL083OWRI': 'WRITE-IN (OTH) State House-83',
 'GSL083RWHE': 'JEFF WHEELAND (REP) State House-83',
 'GSL084DSOS': 'LINDA KAY SOSNIAK (DEM) State House-84',
 'GSL084OWRI': 'WRITE-IN (OTH) State House-84',
 'GSL084REVE': 'GARTH EVERETT (REP) State House-84',
 'GSL085DRAG': 'JENNIFER RAGER-KAY (DEM) State House-85',
 'GSL085OWRI': 'WRITE-IN (OTH) State House-85',
 'GSL085RKEL': 'FRED KELLER (REP) State House-85',
 'GSL086DAND': 'KAREN ANN ANDERSON (DEM) State House-86',
 'GSL086OWRI': 'WRITE-IN (OTH) State House-86',
 'GSL086RKEL': 'MARK K KELLER (REP) State House-86',
 'GSL087DQUI': 'SEAN PATRICK QUINLAN (DEM) State House-87',
 'GSL087OWRI': 'WRITE-IN (OTH) State House-87',
 'GSL087RROT': 'GREG ROTHMAN (REP) State House-87',
 'GSL088DFOS': 'JEAN FOSCHI (DEM) State House-88',
 'GSL088OWRI': 'WRITE-IN (OTH) State House-88',
 'GSL088RDEL': 'SHERYL M DELOZIER (REP) State House-88',
 'GSL089OWRI': 'WRITE-IN (OTH) State House-89',
 'GSL089RKAU': 'ROB KAUFFMAN (REP) State House-89',
 'GSL090OWRI': 'WRITE-IN (OTH) State House-90',
 'GSL090RSCH': 'PAUL SCHEMEL (REP) State House-90',
 'GSL091DQUA': 'MARTY QUALLY (DEM) State House-91',
 'GSL091OWRI': 'WRITE-IN (OTH) State House-91',
 'GSL091RMOU': 'DAN MOUL (REP) State House-91',
 'GSL092DDAN': 'SHANNA DANIELSON (DEM) State House-92',
 'GSL092OWRI': 'WRITE-IN (OTH) State House-92',
 'GSL092RKEE': 'DAWN KEEFER (REP) State House-92',
 'GSL093DRIV': 'DELMA RIVERA-LYTLE (DEM) State House-93',
 'GSL093OWRI': 'WRITE-IN (OTH) State House-93',
 'GSL093RJON': 'MIKE JONES (REP) State House-93',
 'GSL094DSNE': 'STEVE SNELL (DEM) State House-94',
 'GSL094OWRI': 'WRITE-IN (OTH) State House-94',
 'GSL094RSAY': 'STAN SAYLOR (REP) State House-94',
 'GSL095DHIL': 'CAROL HILL-EVANS (DEM) State House-95',
 'GSL095OWRI': 'WRITE-IN (OTH) State House-95',
 'GSL096DSTU': 'MIKE STURLA (DEM) State House-96',
 'GSL096OWRI': 'WRITE-IN (OTH) State House-96',
 'GSL097DGUL': 'DANA HAMP GULICK (DEM) State House-97',
 'GSL097OWRI': 'WRITE-IN (OTH) State House-97',
 'GSL097RMEN': 'STEVE MENTZER (REP) State House-97',
 'GSL098DAUK': 'MARY AUKER-ENDRES (DEM) State House-98',
 'GSL098LMIL': 'JAMES M. MILLER (LIB) State House-98',
 'GSL098OWRI': 'WRITE-IN (OTH) State House-98',
 'GSL098RHIC': 'DAVID S. HICKERNELL (REP) State House-98',
 'GSL099DMAL': 'ELIZABETH MALARKEY (DEM) State House-99',
 'GSL099OWRI': 'WRITE-IN (OTH) State House-99',
 'GSL099RZIM': 'DAVE ZIMMERMAN (REP) State House-99',
 'GSL100DHAM': 'DALE HAMBY (DEM) State House-100',
 'GSL100OWRI': 'WRITE-IN (OTH) State House-100',
 'GSL100RCUT': 'BRYAN D. CUTLER (REP) State House-100',
 'GSL101DLIR': 'CESAR LIRIANO (DEM) State House-101',
 'GSL101OWRI': 'WRITE-IN (OTH) State House-101',
 'GSL101RRYA': 'FRANK RYAN (REP) State House-101',
 'GSL102OWRI': 'WRITE-IN (OTH) State House-102',
 'GSL102RDIA': 'RUSS DIAMOND (REP) State House-102',
 'GSL103DKIM': 'PATTY KIM (DEM) State House-103',
 'GSL103OWRI': 'WRITE-IN (OTH) State House-103',
 'GSL103RHAR': 'ANTHONY THOMAS HARRELL (REP) State House-103',
 'GSL104DSMI': 'PATTY SMITH (DEM) State House-104',
 'GSL104OWRI': 'WRITE-IN (OTH) State House-104',
 'GSL104RHEL': 'SUSAN C. SUE HELM (REP) State House-104',
 'GSL105DEPS': 'ERIC EPSTEIN (DEM) State House-105',
 'GSL105OWRI': 'WRITE-IN (OTH) State House-105',
 'GSL105RLEW': 'ANDREW LEWIS (REP) State House-105',
 'GSL106DLIN': 'JILL LINTA (DEM) State House-106',
 'GSL106OWRI': 'WRITE-IN (OTH) State House-106',
 'GSL106RMEH': 'TOM MEHAFFIE (REP) State House-106',
 'GSL107DDON': 'SARAH DONNELLY (DEM) State House-107',
 'GSL107OWRI': 'WRITE-IN (OTH) State House-107',
 'GSL107RMAS': 'KURT A. MASSER (REP) State House-107',
 'GSL108OWRI': 'WRITE-IN (OTH) State House-108',
 'GSL108RSCH': 'LYNDA J. SCHLEGEL-CULVERLVER (REP) State House-108',
 'GSL109DSAN': 'ED SANDERS (DEM) State House-109',
 'GSL109OWRI': 'WRITE-IN (OTH) State House-109',
 'GSL109RMIL': 'DAVID MILLARD (REP) State House-109',
 'GSL110DIAN': 'DONNA IANNONE (DEM) State House-110',
 'GSL110OWRI': 'WRITE-IN (OTH) State House-110',
 'GSL110RPIC': 'TINA PICKETT (REP) State House-110',
 'GSL111DKER': 'BECKY KERR-KINNEY (DEM) State House-111',
 'GSL111OWRI': 'WRITE-IN (OTH) State House-111',
 'GSL111RFRI': 'JONATHAN FRITZ (REP) State House-111',
 'GSL112DMUL': 'KYLE MULLINS (DEM) State House-112',
 'GSL112OWRI': 'WRITE-IN (OTH) State House-112',
 'GSL112RLEM': 'ERNEST LEMONCELLI (REP) State House-112',
 'GSL113DFLY': 'MARTY FLYNN (DEM) State House-113',
 'GSL113OWRI': 'WRITE-IN (OTH) State House-113',
 'GSL114DKAV': 'SID MICHAELS KAVULICH (DEM) State House-114',
 'GSL114OWRI': 'WRITE-IN (OTH) State House-114',
 'GSL115DMAD': 'MAUREEN MADDEN (DEM) State House-115',
 'GSL115RPAR': 'DAVID PARKER (REP) State House-115',
 'GSL116OWRI': 'WRITE-IN (OTH) State House-116',
 'GSL116RTOO': 'TARAH TOOHIL (REP) State House-116',
 'GSL117LJAS': 'LOU JASIKOFF (LIB) State House-117',
 'GSL117OWRI': 'WRITE-IN (OTH) State House-117',
 'GSL117RBOB': 'KAREN BOBACK (REP) State House-117',
 'GSL118DCAR': 'MIKE CARROLL (DEM) State House-118',
 'GSL118OWRI': 'WRITE-IN (OTH) State House-118',
 'GSL119DMUL': 'GERALD J MULLERY (DEM) State House-119',
 'GSL119OWRI': 'WRITE-IN (OTH) State House-119',
 'GSL119RBEH': 'JUSTIN V BEHRENS (REP) State House-119',
 'GSL120OWRI': 'WRITE-IN (OTH) State House-120',
 'GSL120RKAU': 'AARON KAUFER (REP) State House-120',
 'GSL121DPAS': 'EDDIE DAY PASHINSKI (DEM) State House-121',
 'GSL121OWRI': 'WRITE-IN (OTH) State House-121',
 'GSL121RHEN': 'SUE HENRY (REP) State House-121',
 'GSL122DSCO': 'KARA SCOTT (DEM) State House-122',
 'GSL122OWRI': 'WRITE-IN (OTH) State House-122',
 'GSL122RHEF': 'DOYLE HEFFLEY (REP) State House-122',
 'GSL123DGOO': 'NEAL P. GOODMAN (DEM) State House-123',
 'GSL123OWRI': 'WRITE-IN (OTH) State House-123',
 'GSL124OWRI': 'WRITE-IN (OTH) State House-124',
 'GSL124RKNO': 'JERRY KNOWLES (REP) State House-124',
 'GSL125OWRI': 'WRITE-IN (OTH) State House-125',
 'GSL125RTOB': 'MIKE TOBASH (REP) State House-125',
 'GSL126DROZ': 'MARK ROZZI (DEM) State House-126',
 'GSL126OWRI': 'WRITE-IN (OTH) State House-126',
 'GSL127DCAL': 'TOM CALTAGIRON (DEM) State House-127',
 'GSL127OWRI': 'WRITE-IN (OTH) State House-127',
 'GSL127RGAG': 'VINCENT D. GAGLIARDO (REP) State House-127',
 'GSL128DMET': 'DOUG METCALFE (DEM) State House-128',
 'GSL128OWRI': 'WRITE-IN (OTH) State House-128',
 'GSL128RGIL': 'MARK M. GILLEN (REP) State House-128',
 'GSL129DWER': 'TRISH WERTZ (DEM) State House-129',
 'GSL129OWRI': 'WRITE-IN (OTH) State House-129',
 'GSL129RCOX': 'JIM COX (REP) State House-129',
 'GSL130OWRI': 'WRITE-IN (OTH) State House-130',
 'GSL130RMAL': 'DAVID M. MALONEY (REP) State House-130',
 'GSL131DLEE': 'ANDY LEE (DEM) State House-131',
 'GSL131OWRI': 'WRITE-IN (OTH) State House-131',
 'GSL131RSIM': 'JUSTIN J SIMMONS (REP) State House-131',
 'GSL132DSCH': 'MIKE SCHLOSSBERG (DEM) State House-132',
 'GSL133DMCN': 'JEANNE MCNEILL (DEM) State House-133',
 'GSL134DAPP': 'TOM APPLEBACH (DEM) State House-134',
 'GSL134OWRI': 'WRITE-IN (OTH) State House-134',
 'GSL134RMAC': 'RYAN E MACKENZIE (REP) State House-134',
 'GSL135DSAM': 'STEVE SAMUELSON (DEM) State House-135',
 'GSL135OWRI': 'WRITE-IN (OTH) State House-135',
 'GSL136DFRE': 'ROBERT FREEMAN (DEM) State House-136',
 'GSL136OWRI': 'WRITE-IN (OTH) State House-136',
 'GSL137DCOZ': 'AMY COZZE (DEM) State House-137',
 'GSL137LREA': 'ED REAGAN (LIB) State House-137',
 'GSL137OWRI': 'WRITE-IN (OTH) State House-137',
 'GSL137REMR': 'JOE EMRICK (REP) State House-137',
 'GSL138DDON': 'DEAN DONAHER (DEM) State House-138',
 'GSL138LTOW': 'JAKE TOWNE (LIB) State House-138',
 'GSL138OWRI': 'WRITE-IN (OTH) State House-138',
 'GSL138RHAH': 'MARCIA HAHN (REP) State House-138',
 'GSL139DMAR': 'ORLANDO MARRERO (DEM) State House-139',
 'GSL139RPEI': 'MIKE PEIFER (REP) State House-139',
 'GSL140DGAL': 'JOHN T. GALLOWAY (DEM) State House-140',
 'GSL141DDAV': 'TINA DAVIS (DEM) State House-141',
 'GSL141RSPO': 'ANTHONY J. SPOSATO (REP) State House-141',
 'GSL142DLAR': 'LAUREN LAREAU (DEM) State House-142',
 'GSL142RFAR': 'FRANK FARRY (REP) State House-142',
 'GSL143DULL': 'WENDY ULLMAN (DEM) State House-143',
 'GSL143RFLO': 'JOE FLOOD (REP) State House-143',
 'GSL144DBUC': 'MEREDITH BUCK (DEM) State House-144',
 'GSL144RPOL': 'TODD POLINCHOCK (REP) State House-144',
 'GSL145DKLI': 'BRIAN KLINE (DEM) State House-145',
 'GSL145RSTA': 'CRAIG STAATS (REP) State House-145',
 'GSL146DCIR': 'JOE CIRESI (DEM) State House-146',
 'GSL146OWRI': 'WRITE-IN (OTH) State House-146',
 'GSL146RQUI': 'THOMAS J QUIGLEY (REP) State House-146',
 'GSL147DCAM': 'JOSH CAMSON (DEM) State House-147',
 'GSL147OWRI': 'WRITE-IN (OTH) State House-147',
 'GSL147RTOE': 'MARCY TOEPEL (REP) State House-147',
 'GSL148DDAL': 'MARY JO DALEY (DEM) State House-148',
 'GSL148OWRI': 'WRITE-IN (OTH) State House-148',
 'GSL149DBRI': 'TIM BRIGGS (DEM) State House-149',
 'GSL149OWRI': 'WRITE-IN (OTH) State House-149',
 'GSL150DWEB': 'JOE WEBSTER (DEM) State House-150',
 'GSL150OWRI': 'WRITE-IN (OTH) State House-150',
 'GSL150RFOU': 'NICK FOUNTAIN (REP) State House-150',
 'GSL151DROT': 'SARA JOHNSON ROTHMAN (DEM) State House-151',
 'GSL151OWRI': 'WRITE-IN (OTH) State House-151',
 'GSL151RSTE': 'TODD STEPHENS (REP) State House-151',
 'GSL152DBOL': 'DARYL BOLING (DEM) State House-152',
 'GSL152OWRI': 'WRITE-IN (OTH) State House-152',
 'GSL152RMUR': 'THOMAS P. MURT (REP) State House-152',
 'GSL153DSAN': 'BEN SANCHEZ (DEM) State House-153',
 'GSL153LBOZ': 'MARC BOZZACCO (LIB) State House-153',
 'GSL153OWRI': 'WRITE-IN (OTH) State House-153',
 'GSL153RBEA': 'DOUGLAS BEAVER (REP) State House-153',
 'GSL154DMCC': 'STEVE MCCARTER (DEM) State House-154',
 'GSL154OWRI': 'WRITE-IN (OTH) State House-154',
 'GSL154RBOW': 'KATHY GARRY BOWERS (REP) State House-154',
 'GSL155DOTT': 'DANIELLE FRIEL OTTEN (DEM) State House-155',
 'GSL155OWRI': 'WRITE-IN (OTH) State House-155',
 'GSL155RCOR': 'BECKY CORBIN (REP) State House-155',
 'GSL156DCOM': 'CAROLYN COMITTA (DEM) State House-156',
 'GSL156OWRI': 'WRITE-IN (OTH) State House-156',
 'GSL156RDEM': 'NICHOLAS J DEMINSKI (REP) State House-156',
 'GSL157DSHU': 'MELISSA SHUSTERMAN (DEM) State House-157',
 'GSL157OWRI': 'WRITE-IN (OTH) State House-157',
 'GSL157RKAM': 'WARREN KAMPF (REP) State House-157',
 'GSL158DSAP': 'CHRISTINA SAPPEY (DEM) State House-158',
 'GSL158OWRI': 'WRITE-IN (OTH) State House-158',
 'GSL158RROE': 'ERIC ROE (REP) State House-158',
 'GSL159DKIR': 'BRIAN KIRKLAND (DEM) State House-159',
 'GSL159OWRI': 'WRITE-IN (OTH) State House-159',
 'GSL159RMOT': 'RUTH MOTON (REP) State House-159',
 'GSL160DAND': 'ANTON ANDREW (DEM) State House-160',
 'GSL160OWRI': 'WRITE-IN (OTH) State House-160',
 'GSL160RBAR': 'STEPHEN BARRAR (REP) State House-160',
 'GSL161DKRU': 'LEANNE KRUEGER-BRANEKY (DEM) State House-161',
 'GSL161OWRI': 'WRITE-IN (OTH) State House-161',
 'GSL161RMOR': 'PATTI RODGERS MORRISETTE (REP) State House-161',
 'GSL162DDEL': 'DAVE DELLOSO (DEM) State House-162',
 'GSL162OWRI': 'WRITE-IN (OTH) State House-162',
 'GSL162RHOP': 'MARY HOPPER (REP) State House-162',
 'GSL163DZAB': 'MIKE ZABEL (DEM) State House-163',
 'GSL163OWRI': 'WRITE-IN (OTH) State House-163',
 'GSL163RSAN': 'JAMES SANTORA (REP) State House-163',
 'GSL164DDAV': 'MARGO L. DAVIDSON (DEM) State House-164',
 'GSL164OWRI': 'WRITE-IN (OTH) State House-164',
 'GSL164RBAI': 'INDER BAINS (REP) State House-164',
 'GSL165DOMA': "JENNIFER O'MARA (DEM) State House-165",
 'GSL165OWRI': 'WRITE-IN (OTH) State House-165',
 'GSL165RCHA': 'ALEXANDER T. CHARLTON (REP) State House-165',
 'GSL166DVIT': 'GREG VITALI (DEM) State House-166',
 'GSL166OWRI': 'WRITE-IN (OTH) State House-166',
 'GSL166RRUB': 'BALTAZAR RUBIO (REP) State House-166',
 'GSL167DHOW': 'KRISTINE HOWARD (DEM) State House-167',
 'GSL167OWRI': 'WRITE-IN (OTH) State House-167',
 'GSL167RMIL': 'DUANE MILNE (REP) State House-167',
 'GSL168DSEA': 'KRISTIN SEALE (DEM) State House-168',
 'GSL168OWRI': 'WRITE-IN (OTH) State House-168',
 'GSL168RQUI': 'CHRIS QUINN (REP) State House-168',
 'GSL169DHAM': 'SARAH E HAMMOND (DEM) State House-169',
 'GSL169OWRI': 'WRITE-IN (OTH) State House-169',
 'GSL169RKLU': 'KATE A KLUNK (REP) State House-169',
 'GSL170DDOY': 'MIKE DOYLE (DEM) State House-170',
 'GSL170OWRI': 'WRITE-IN (OTH) State House-170',
 'GSL170RWHI': 'MARTINA WHITE (REP) State House-170',
 'GSL171DMCC': 'ERIN MCCRACKEN (DEM) State House-171',
 'GSL171OWRI': 'WRITE-IN (OTH) State House-171',
 'GSL171RBEN': 'KERRY A BENNINGHOFF (REP) State House-171',
 'GSL172DBOY': 'KEVIN J. BOYLE (DEM) State House-172',
 'GSL172OWRI': 'WRITE-IN (OTH) State House-172',
 'GSL173DDRI': 'MIKE DRISCOLL (DEM) State House-173',
 'GSL173OWRI': 'WRITE-IN (OTH) State House-173',
 'GSL174DNEI': 'ED NEILSON (DEM) State House-174',
 'GSL174OWRI': 'WRITE-IN (OTH) State House-174',
 'GSL175DISA': 'MARYLOUISE ISAACSON (DEM) State House-175',
 'GSL175OWRI': 'WRITE-IN (OTH) State House-175',
 'GSL176DWIL': 'CLAUDETTE WILLIAMS (DEM) State House-176',
 'GSL176RRAD': 'JACK RADER (REP) State House-176',
 'GSL177DHOH': 'JOE HOHENSTEIN (DEM) State House-177',
 'GSL177OWRI': 'WRITE-IN (OTH) State House-177',
 'GSL177RKOZ': 'PATTY KOZLOWSKI (REP) State House-177',
 'GSL178DTAI': 'HELEN TAI (DEM) State House-178',
 'GSL178RTHO': 'WENDI THOMAS (REP) State House-178',
 'GSL179DDAW': 'JASON T. DAWKINS (DEM) State House-179',
 'GSL179OWRI': 'WRITE-IN (OTH) State House-179',
 'GSL180DCRU': 'ANGEL CRUZ (DEM) State House-180',
 'GSL180OWRI': 'WRITE-IN (OTH) State House-180',
 'GSL181DKEN': 'MALCOLM KENYATTA (DEM) State House-181',
 'GSL181OWRI': 'WRITE-IN (OTH) State House-181',
 'GSL181RSTR': 'MILTON STREET (REP) State House-181',
 'GSL182DSIM': 'BRIAN SIMS (DEM) State House-182',
 'GSL182IMCD': 'JAMES MCDEVITT (IND) State House-182',
 'GSL182OWRI': 'WRITE-IN (OTH) State House-182',
 'GSL183DRUF': 'JASON RUFF (DEM) State House-183',
 'GSL183OWRI': 'WRITE-IN (OTH) State House-183',
 'GSL183RMAK': 'ZACH MAKO (REP) State House-183',
 'GSL184DFIE': 'ELIZABETH FIEDLER (DEM) State House-184',
 'GSL184OWRI': 'WRITE-IN (OTH) State House-184',
 'GSL185DDON': 'MARIA P DONATUCCI (DEM) State House-185',
 'GSL185OWRI': 'WRITE-IN (OTH) State House-185',
 'GSL186DHAR': 'JORDAN A. HARRIS (DEM) State House-186',
 'GSL186OWRI': 'WRITE-IN (OTH) State House-186',
 'GSL187DBLI': 'MICHAEL BLICHAR JR (DEM) State House-187',
 'GSL187OWRI': 'WRITE-IN (OTH) State House-187',
 'GSL187RDAY': 'GARY DAY (REP) State House-187',
 'GSL188DROE': 'JAMES R. ROEBUCK (DEM) State House-188',
 'GSL188OWRI': 'WRITE-IN (OTH) State House-188',
 'GSL189DROD': 'ADAM RODRIGUEZ (DEM) State House-189',
 'GSL189RBRO': 'ROSEMARY BROWN (REP) State House-189',
 'GSL190DBRO': 'VANESSA L. BROWN (DEM) State House-190',
 'GSL190OWRI': 'WRITE-IN (OTH) State House-190',
 'GSL191DMCC': 'JOANNA E. MCCLINTON (DEM) State House-191',
 'GSL191OWRI': 'WRITE-IN (OTH) State House-191',
 'GSL192DCEP': 'MORGAN CEPHAS (DEM) State House-192',
 'GSL192OWRI': 'WRITE-IN (OTH) State House-192',
 'GSL193DNEL': 'MATTHEW NELSON (DEM) State House-193',
 'GSL193OWRI': 'WRITE-IN (OTH) State House-193',
 'GSL193RECK': 'TORREN ECKER (REP) State House-193',
 'GSL194DDEL': 'PAMELA DELISSIO (DEM) State House-194',
 'GSL194LBAL': 'MATT BALTSAR (LIB) State House-194',
 'GSL194OWRI': 'WRITE-IN (OTH) State House-194',
 'GSL194RSTE': 'SEAN STEVENS (REP) State House-194',
 'GSL195DBUL': 'DONNA BULLOCK (DEM) State House-195',
 'GSL195OWRI': 'WRITE-IN (OTH) State House-195',
 'GSL196OWRI': 'WRITE-IN (OTH) State House-196',
 'GSL196RGRO': 'SETH GROVE (REP) State House-196',
 'GSL197DBUR': 'DANILO BURGOS (DEM) State House-197',
 'GSL197OWRI': 'WRITE-IN (OTH) State House-197',
 'GSL198DYOU': 'ROSITA C YOUNGBLOOD (DEM) State House-198',
 'GSL198OWRI': 'WRITE-IN (OTH) State House-198',
 'GSL199DMCG': 'SHERWOOD MCGINNIS (DEM) State House-199',
 'GSL199LBOU': 'CHARLES M BOUST (LIB) State House-199',
 'GSL199OWRI': 'WRITE-IN (OTH) State House-199',
 'GSL199RGLE': 'BARBARA GLEIM (REP) State House-199',
 'GSL200DRAB': 'CHRIS RABB (DEM) State House-200',
 'GSL200OWRI': 'WRITE-IN (OTH) State House-200',
 'GSL201DKIN': 'STEPHEN KINSEY (DEM) State House-201',
 'GSL201OWRI': 'WRITE-IN (OTH) State House-201',
 'GSL202DSOL': 'JARED SOLOMON (DEM) State House-202',
 'GSL202OWRI': 'WRITE-IN (OTH) State House-202',
 'GSL203DFIT': 'ISABELLA FITZGERALD (DEM) State House-203',
 'GSL203OWRI': 'WRITE-IN (OTH) State House-203',
 'GSU02DTAR': 'CHRISTINE M. TARTAGLIONE (DEM) State Senate-2',
 'GSU02OWRI': 'WRITE-IN (OTH) State Senate-2',
 'GSU04DHAY': 'ART HAYWOOD (DEM) State Senate-4',
 'GSU04OWRI': 'WRITE-IN (OTH) State Senate-4',
 'GSU04RHOL': 'RON HOLT (REP) State Senate-4',
 'GSU06DDAV': 'TINA DAVIS (DEM) State Senate-6',
 'GSU06RTOM': 'ROBERT TOMMY TOMLINSON (REP) State Senate-6',
 'GSU08DWIL': 'ANTHONY HARDY WILLIAMS (DEM) State Senate-8',
 'GSU08OWRI': 'WRITE-IN (OTH) State Senate-8',
 'GSU10DSAN': 'STEVE SANTARSIERO (DEM) State Senate-10',
 'GSU10RQUI': 'MARGUERITE QUINN (REP) State Senate-10',
 'GSU12DCOL': 'MARIA COLLETT (DEM) State Senate-12',
 'GSU12OWRI': 'WRITE-IN (OTH) State Senate-12',
 'GSU12RGRE': 'STEWART GREENLEAF JR (REP) State Senate-12',
 'GSU14DYUD': 'JOHN T YUDICHAK (DEM) State Senate-14',
 'GSU14OWRI': 'WRITE-IN (OTH) State Senate-14',
 'GSU16DPIN': 'MARK PINSLEY (DEM) State Senate-16',
 'GSU16RBRO': 'PAT BROWNE (REP) State Senate-16',
 'GSU18DBOS': 'LISA BOSCOLA (DEM) State Senate-18',
 'GSU18OWRI': 'WRITE-IN (OTH) State Senate-18',
 'GSU20GSWE': 'JOHN J SWEENEY (GRN) State Senate-20',
 'GSU20OWRI': 'WRITE-IN (OTH) State Senate-20',
 'GSU20RBAK': 'LISA BAKER (REP) State Senate-20',
 'GSU22DBLA': 'JOHN P BLAKE (DEM) State Senate-22',
 'GSU22OWRI': 'WRITE-IN (OTH) State Senate-22',
 'GSU22RSCA': 'FRANK SCAVO (REP) State Senate-22',
 'GSU24DFIE': 'LINDA FIELDS (DEM) State Senate-24',
 'GSU24OWRI': 'WRITE-IN (OTH) State Senate-24',
 'GSU24RMEN': 'BOB MENSCH (REP) State Senate-24',
 'GSU26DKEA': 'TIM KEARNEY (DEM) State Senate-26',
 'GSU26OWRI': 'WRITE-IN (OTH) State Senate-26',
 'GSU26RMCG': 'THOMAS J. MCGARRIGLE SR (REP) State Senate-26',
 'GSU28DHIG': 'JUDITH HIGGINS (DEM) State Senate-28',
 'GSU28OWRI': 'WRITE-IN (OTH) State Senate-28',
 'GSU28RPHI': 'KRISTIN PHILLIPS-HILL (REP) State Senate-28',
 'GSU30DBES': 'EMILY BEST (DEM) State Senate-30',
 'GSU30OWRI': 'WRITE-IN (OTH) State Senate-30',
 'GSU30RWAR': 'JUDY WARD (REP) State Senate-30',
 'GSU32DGER': 'PAM GERARD (DEM) State Senate-32',
 'GSU32OWRI': 'WRITE-IN (OTH) State Senate-32',
 'GSU32RSTE': 'PAT STEFANO (REP) State Senate-32',
 'GSU34DNAN': 'EZRA NANES (DEM) State Senate-34',
 'GSU34OWRI': 'WRITE-IN (OTH) State Senate-34',
 'GSU34RCOR': 'JACOB D CORMAN III (REP) State Senate-34',
 'GSU36DTRO': 'BILL TROUTMAN (DEM) State Senate-36',
 'GSU36OWRI': 'WRITE-IN (OTH) State Senate-36',
 'GSU36RAUM': 'RYAN P. AUMENT (REP) State Senate-36',
 'GSU38DWIL': 'LINDSEY WILLIAMS (DEM) State Senate-38',
 'GSU38OWRI': 'WRITE-IN (OTH) State Senate-38',
 'GSU38RSHA': 'JEREMY SHAFFER (REP) State Senate-38',
 'GSU40DPRO': 'TARAH PROBST (DEM) State Senate-40',
 'GSU40LREI': 'ADAM REINHARDT (LIB) State Senate-40',
 'GSU40OWRI': 'WRITE-IN (OTH) State Senate-40',
 'GSU40RSCA': 'MARIO SCAVELLO (REP) State Senate-40',
 'GSU42DFON': 'WAYNE D. FONTANA (DEM) State Senate-42',
 'GSU42OWRI': 'WRITE-IN (OTH) State Senate-42',
 'GSU44DMUT': 'KATIE MUTH (DEM) State Senate-44',
 'GSU44OWRI': 'WRITE-IN (OTH) State Senate-44',
 'GSU44RRAF': 'JOHN RAFFERTY (REP) State Senate-44',
 'GSU46DCRA': 'JAMES R. CRAIG (DEM) State Senate-46',
 'GSU46OWRI': 'WRITE-IN (OTH) State Senate-46',
 'GSU46RBAR': 'CAMERA BARTOLOTTA (REP) State Senate-46',
 'GSU48DHER': 'LOIS HERR (DEM) State Senate-48',
 'GSU48OWRI': 'WRITE-IN (OTH) State Senate-48',
 'GSU48RFOL': 'MIKE FOLMER (REP) State Senate-48',
 'GSU50DMUL': 'SUE MULVEY (DEM) State Senate-50',
 'GSU50OWRI': 'WRITE-IN (OTH) State Senate-50',
 'GSU50RBRO': 'MICHELE BROOKS (REP) State Senate-50',
 'SCON07DSCA': 'MARY GAY SCANLON (DEM) Special U.S. House-7',
 'SCON07GJOH': 'BRIANNA JOHNSTON (GRN) Special U.S. House-7',
 'SCON07LSAL': 'SANDRA TERESA SALAS (LIB) Special U.S. House-7',
 'SCON07OWRI': 'WRITE-IN (OTH) Special U.S. House-7',
 'SCON07RKIM': 'PEARL KIM (REP) Special U.S. House-7',
 'SCON15DWIL': 'SUSAN WILD (DEM) Special U.S. House-15',
 'SCON15LSIL': 'TIM SILFIES (LIB) Special U.S. House-15',
 'SCON15OWRI': 'WRITE-IN (OTH) Special U.S. House-15',
 'SCON15RNOT': 'MARTY NOTHSTEIN (REP) Special U.S. House-15'}

In [30]:
import os

fields_dict_0 = pa_cand_dict
race_field_header_0 = "***mi_20_st_prec.zip***"


#sldl_holder = pd.read_csv("./full_cand_dict.csv")
#fields_dict_1 = dict(zip(sldl_holder["Candidate"],sldl_holder["Column"]))
add_race_field_header_1 = "***mi_20_sldl_prec.zip***)"

#cong_holder = pd.read_csv("./full_cand_dict.csv")
#fields_dict_2 = dict(zip(cong_holder["Candidate"],cong_holder["Column"]))
add_race_field_header_2 = "***mi_20_cong_prec.zip***"

In [31]:
fields_dict_0["UNIQUE_ID"] = "Unique Identifier"
fields_dict_0['COUNTYFP'] = "County FIP"
fields_dict_0['STATEFP'] = "State FIP"
fields_dict_0["VTDST"] = "VTD Code"
fields_dict_0["NAME"] = "VTD Name"
fields_dict_0["COUNTY"] = "County Name"
fields_dict_0["PREC_NAME"] = "Precinct Name"

In [32]:
fields_dict = {}
fields_dict["UNIQUE_ID"] = "Unique Identifier"
fields_dict['COUNTYFP'] = "County FIP"
fields_dict['STATEFP'] = "State FIP"
fields_dict["VTDST"] = "VTD Code"
fields_dict["NAME"] = "VTD Name"
fields_dict["COUNTY"] = "County Name"
fields_dict["PREC_NAME"] = "Precinct Name"

fields_dict["CONG_DIST"] = "U.S. Congressional District (n/a if no votes in one of these races)"
fields_dict["SCONG_DIST"] = "U.S. Congressional District Special Election (2013-2018 map - n/a if no votes in one of these races)"
fields_dict["SLDU_DIST"] = "State Senate District (n/a if no votes in one of these races)"
fields_dict["SLDL_DIST"] = "State Representative District (n/a if no votes in one of these races)"

In [33]:
office_dict = {}
office_dict["SLXX"] = 'State Legislative Lower'
office_dict["SUXX"] = 'State Legislative Upper'
office_dict["CONXX"] = 'U.S. Congress'
office_dict['PRE'] = 'President'
office_dict["USS"] = 'U.S. Senator'
office_dict["GOV"] = 'Governor'




In [34]:
office_codes = office_dict

In [35]:
full_readme = erj_readme_template(fields_dict, stateabrv, state, year, election_type, additional_races, retrieval_date, upload_date, vest_file_link, 
                        raw_data_source, state_erj_repo, office_codes, additional_notes)

In [37]:
full_readme = erj_readme_template(stateabrv, state, year, election_type, additional_races, retrieval_date, vest_file_link, 
                        raw_data_source, state_erj_repo,
                        office_codes, 
                        race_field_header_0, fields_dict_0,
                       additional_notes=' ')

In [39]:
export_readme("./pa_gen_18_prec/README.txt", "Pennsylvania", "general", full_readme)

Pennsylvania general  readme moved to folder


In [41]:
main_file = gp.read_file("/Users/peterhorton/Documents/RDH/erj/erj-pa/2018/general/pa_gen_18_prec/pa_st_sldu_spec_cong_18_prec.shp")

In [42]:
main_file.columns

Index(['UNIQUE_ID', 'COUNTYFP', 'STATEFP', 'VTDST', 'NAME', 'COUNTY',
       'PREC_NAME', 'SCONG_DIST', 'SLDU_DIST', 'G18GOVDWOL', 'G18GOVGGLO',
       'G18GOVLKRA', 'G18GOVOWRI', 'G18GOVRWAG', 'G18USSDCAS', 'G18USSGGAL',
       'G18USSLKER', 'G18USSOWRI', 'G18USSRBAR', 'GSU02DTAR', 'GSU02OWRI',
       'GSU04DHAY', 'GSU04OWRI', 'GSU04RHOL', 'GSU06DDAV', 'GSU06RTOM',
       'GSU08DWIL', 'GSU08OWRI', 'GSU10DSAN', 'GSU10RQUI', 'GSU12DCOL',
       'GSU12OWRI', 'GSU12RGRE', 'GSU14DYUD', 'GSU14OWRI', 'GSU16DPIN',
       'GSU16RBRO', 'GSU18DBOS', 'GSU18OWRI', 'GSU20GSWE', 'GSU20OWRI',
       'GSU20RBAK', 'GSU22DBLA', 'GSU22OWRI', 'GSU22RSCA', 'GSU24DFIE',
       'GSU24OWRI', 'GSU24RMEN', 'GSU26DKEA', 'GSU26OWRI', 'GSU26RMCG',
       'GSU28DHIG', 'GSU28OWRI', 'GSU28RPHI', 'GSU30DBES', 'GSU30OWRI',
       'GSU30RWAR', 'GSU32DGER', 'GSU32OWRI', 'GSU32RSTE', 'GSU34DNAN',
       'GSU34OWRI', 'GSU34RCOR', 'GSU36DTRO', 'GSU36OWRI', 'GSU36RAUM',
       'GSU38DWIL', 'GSU38OWRI', 'GSU38RSHA', 'GSU40DPR